In [1]:
import sys
!{sys.executable} -m pip install scipy==1.5.2

In [11]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [12]:
from itertools import count # 迭代器
from collections import defaultdict # 使用dict时，如果引用的Key不存在，就会抛出KeyError。如果希望key不存在时，返回一个默认值，就可以用defaultdict
from scipy.sparse import csr # csr_matrix，全名为Compressed Sparse Row，是按行对矩阵进行压缩的。CSR需要三类数据：数值，列号，以及行偏移量。CSR是一种编码的方式，其中，数值与列号的含义，与coo里是一致的。行偏移表示某一行的第一个元素在values里面的起始偏移位置。 
import numpy as np
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
#import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from tqdm import tqdm_notebook as tqdm # 可以显示循环的进度条的库

Instructions for updating:
non-resource variables are not supported in the long term


In [96]:
item_features = pd.read_csv("preproceed_data/item_features.csv", header = None)
user_features = pd.read_csv("preproceed_data/user_features.csv", header = None)
item_features

,0,1,2,3,4,5,6,7
0,0.0,12.369674,21.918972,0.0,0.0,0.000000,4.377610,0.000000
1,0.0,7.135008,8.602298,0.0,0.0,0.000000,3.646132,0.000000
2,0.0,3.433517,2.322647,0.0,0.0,0.000000,1.801508,0.000000
3,0.0,5.650490,1.497254,0.0,0.0,0.000000,0.000000,0.000000
4,0.0,3.414955,0.000000,0.0,0.0,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...
1677,0.0,0.000000,0.000000,0.0,0.0,943.701782,0.000000,1161.409180
1678,0.0,0.000000,0.000000,0.0,0.0,944.199463,0.000000,1162.020874
1679,0.0,0.000000,0.000000,0.0,0.0,944.601929,0.000000,1162.517822
1680,0.0,0.000000,0.000000,0.0,0.0,945.498108,0.000000,1163.618286


In [ ]:
observation_matrix = []
for u in range(user_features.shape[0]):
    for i in range(item_features.shape[0]):
        user_f = user_features[u]
        item_f = item_features[i]
        observed = np.concatenate((user_f, item_f))
        observation_matrix.append(observed)
observation_matrix = np.array(observation_matrix)

In [94]:
def vectorize_dic(dic, ix=None, p=None):
    """ 
    Creates a scipy csr matrix from a list of lists (each inner list is a set of values corresponding to a feature) 
    
    parameters:
    -----------
    dic -- dictionary of feature lists. Keys are the name of features
    ix -- index generator (default None)
    p -- dimension of featrure space (number of columns in the sparse matrix) (default None)
    """
    if (ix == None):
        d = count(0)
        ix = defaultdict(lambda: next(d)) 
        
    n = len(list(dic.values())[0]) # num samples
    g = len(list(dic.keys())) # num groups
    nz = n * g # number of non-zeros

    col_ix = np.empty(nz, dtype=int)     
    
    i = 0
    for k, lis in dic.items():     
        # append index el with k in order to prevet mapping different columns with same id to same index
        col_ix[i::g] = [ix[str(el) + str(k)] for el in lis]
        i += 1
        
    row_ix = np.repeat(np.arange(0, n), g)      
    data = np.ones(nz)
    
    if (p == None):
        p = len(ix)
        
    ixx = np.where(col_ix < p)

    return csr.csr_matrix((data[ixx],(row_ix[ixx], col_ix[ixx])), shape=(n, p)), ix

cols = ['user','item','rating','timestamp']



#train = pd.read_csv('preproceed_data/item_features.csv',delimiter='\t',names = cols)
#test = pd.read_csv('preproceed_data/rat',delimiter='\t',names = cols)

x_train,ix = vectorize_dic({'users':train['user'].values,  'items':train['item'].values},n=len(train.index),g=2)

#x_test,ix = vectorize_dic({'users':test['user'].values,   'items':test['item'].values},ix,x_train.shape[1],n=len(test.index),g=2)

x_train,ix = vectorize_dic({'users':train['user'].values,  'items':train['item'].values})

#x_test,ix = vectorize_dic({'users':test['user'].values,   'items':test['item'].values},ix,x_train.shape[1])

print(x_train)
y_train = train['rating'].values
#y_test = test['rating'].values

x_train = x_train.todense() # toarray returns an ndarray; todense returns a matrix. If you want a matrix, use todense otherwise, use toarray
#x_test = x_test.todense()


KeyError: 8

In [66]:
def batcher(X_, y_=None, batch_size=-1):
    n_samples = X_.shape[0]

    if batch_size == -1:
        batch_size = n_samples
    if batch_size < 1:
       raise ValueError('Parameter batch_size={} is unsupported'.format(batch_size))

    for i in range(0, n_samples, batch_size):
        upper_bound = min(i + batch_size, n_samples)
        ret_x = X_[i:upper_bound]
        ret_y = None
        if y_ is not None:
            ret_y = y_[i:i + batch_size]
            yield (ret_x, ret_y)


In [68]:
n,p = x_train.shape

k = 10

x = tf.placeholder('float',[None,p])

y = tf.placeholder('float',[None,1])

w0 = tf.Variable(tf.zeros([1]))
w = tf.Variable(tf.zeros([p]))

v = tf.Variable(tf.random_normal([k,p],mean=0,stddev=0.01))

y_hat = tf.Variable(tf.zeros([n,1]))

linear_terms = tf.add(w0,tf.reduce_sum(tf.multiply(w,x),1,keep_dims=True)) # n * 1
pair_interactions = 0.5 * tf.reduce_sum(
    tf.subtract(
        tf.pow(
            tf.matmul(x,tf.transpose(v)),2),
        tf.matmul(tf.pow(x,2),tf.transpose(tf.pow(v,2)))
    ),axis = 1 , keep_dims=True)

y_hat = tf.add(linear_terms,pair_interactions)

In [70]:
lambda_w = tf.constant(0.001,name='lambda_w')
lambda_v = tf.constant(0.001,name='lambda_v')

l2_norm = tf.reduce_sum(
    tf.add(
        tf.multiply(lambda_w,tf.pow(w,2)),
        tf.multiply(lambda_v,tf.pow(v,2))
    )
)

error = tf.reduce_mean(tf.square(y-y_hat))
loss = tf.add(error,l2_norm)

train_op = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(loss)


In [87]:
epochs = 10
batch_size = 1000

# Launch the graph
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)

    for epoch in tqdm(range(epochs), unit='epoch'):
        perm = np.random.permutation(x_train.shape[0]) # 函数shuffle与permutation都是对原来的数组进行重新洗牌（即随机打乱原来的元素顺序）；区别在于shuffle直接在原来的数组上进行操作，改变原来数组的顺序，无返回值。而permutation不直接在原来的数组上进行操作，而是返回一个新的打乱顺序的数组，并不改变原来的数组。
        # iterate over batches
        for bX, bY in batcher(x_train[perm], y_train[perm], batch_size):
            #_,t = sess.run([train_op,loss], feed_dict={x: bX.reshape(-1, p), y: bY.reshape(-1, 1)})
            s = sess.run(y_hat, feed_dict={x: bX.reshape(n,p), y: bY.reshape(n,1)})
            #print(t)
            print(s)

    errors = []
    pred = []
    for bX, bY in batcher(x_test, y_test):
        errors.append(sess.run(error, feed_dict={x: bX.reshape(-1, p), y: bY.reshape(-1, 1)}))
        print(errors)
    RMSE = np.sqrt(np.array(errors).mean())
    print (RMSE)


/Users/liuziyu/opt/anaconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if __name__ == '__main__':


  0%|          | 0/10 [00:00<?, ?epoch/s]

[[ 3.21617059e-04]
 [ 2.16137181e-04]
 [ 1.68411148e-04]
 [ 3.00693675e-04]
 [ 3.71274771e-04]
 [-2.54028681e-04]
 [-1.28694926e-04]
 [-1.73743829e-04]
 [-3.56330944e-04]
 [-2.63978363e-05]
 [ 4.80282702e-04]
 [-2.20524962e-05]
 [ 6.54609539e-05]
 [-1.80310599e-04]
 [ 7.57084112e-04]
 [-5.40696899e-04]
 [-5.05748103e-05]
 [ 3.81681195e-04]
 [-4.67650127e-04]
 [-5.26701187e-05]
 [ 3.90801957e-04]
 [ 2.56649771e-04]
 [ 5.46868978e-05]
 [-1.97308866e-04]
 [ 2.05120567e-04]
 [ 3.83344268e-05]
 [ 5.85765374e-05]
 [-2.56303174e-04]
 [ 5.06266442e-05]
 [-2.90367985e-04]
 [-1.49271596e-04]
 [-1.84941964e-05]
 [-5.08683588e-05]
 [-2.48546014e-04]
 [ 2.01230578e-05]
 [-4.60993215e-05]
 [-1.24649174e-04]
 [-4.56018432e-04]
 [ 3.55516589e-04]
 [ 1.86238176e-04]
 [-3.00191838e-04]
 [ 3.11002601e-04]
 [ 3.34986660e-04]
 [-3.59780621e-04]
 [ 8.07443212e-05]
 [-4.75410838e-04]
 [-2.59629451e-06]
 [ 3.63139086e-04]
 [ 3.51404597e-04]
 [-3.12179909e-05]
 [ 1.83382435e-04]
 [ 3.04583682e-05]
 [ 2.2940491

ValueError: cannot reshape array of size 24734890 into shape (945)